In [1]:
from IPython import get_ipython
import numpy as np
import pandas as pd
# pd.set_option('display.max_columns', 1000)
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
# from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import gc
import json
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score

%matplotlib inline

%load_ext autoreload
%autoreload 2
from DSB_func_ishikawa import *

import warnings
warnings.filterwarnings('ignore')



C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
#・event_idは、あるユーザのあるゲームに対して一つ割り当てられる。
#  一度閉じてもevent_idは変更されない、game_sessionは変更される。
#・game_sessionは、ゲームの種別ごとの、毎回のプレイ(開いてから閉じるまで)に与えられるIDであり
#  一度閉じると同じユーザであっても変更される。
#・timestampはtimestampだがstring型なので変換が必要
#・event_dataは謎のjson
#・installation_idでユーザを区別している。
#・event_countはevent_dataから抽出されるゲームセッション内の増加値で、1が引かれている。
#・event_codeはそれぞれのtitleで固有のイベントの種類だが、2000は常にゲームスタートを表す。
#・game_timeはゲーム開始からの秒数(ms)
#・titleはゲームタイトル
#・typeはゲームまたはビデオのタイプ
#  'Game', 'Assessment(評価)', 'Activity(活動？)', 'Clip(ビデオクリップ？)'
#・worldはゲームやビデオが属するセクション、 
#  'NONE' (at the app's start screen), TREETOPCITY' (Length/Height),
#  'MAGMAPEAK' (Capacity/Displacement), 'CRYSTALCAVES' (Weight)

# それぞれのinstallation_idの最後のAssesment群について、
# そのAssesmentが
# 3:最初のトライでtrue
# 2:二回目のトライでtrue
# 1:三回目以降のトライでtrue
# 0:正解できなかった
# を最終的に予測する
# そのためには最後のAssesmentの最終トライでcorrectがtrueかどうかを確認(num_correct)し
# その後、そのAssessment群でいくつ不正解したかを確認(num_incorrect)する。
# そこから最終的なAccuracy_groupを算出する

# Bird MeasurerのAssessmentは二段階で、4100のあとに4110が続く。この場合予測するのは4110の方。
# 他は4100一回のみ

#Assessmentとそれ以外で指標を分ける

#全部横に並べて入れるか、最終try以前のデータから作った特徴量を付加した1行で予測をするか
#installation_idごとの最後のAssessment群
#今までのAssessmentのTrue回数

#testデータのそれぞれのinstallation_idの最後のトランザクションは常に2000でAssessmentが始まる
#その結果を予測する

#その子供が最後の2000で始まるAssessmentで良い結果を残すにはどういう条件が必要か

#過去に同じAssessmentをプレイしている 0, 1
#過去に同じAssessmentでtrueである 0, 1
#今までのtransaction数
#今までのAssessmentのプレイ回数(gamessessionのうちAssessmentをカウント)

#これまでのトランザクションのタイムスタンプ間の差の中央値＝速度

#gamesessionで区切ると、閉じて再起動してすぐプレイしたときにバグる
#以前のもの全てで入れるしかない


In [3]:
print('Loading')
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
labels = pd.read_csv('./input/train_labels.csv')
sub = pd.read_csv('./input/sample_submission.csv')


Loading


In [4]:
# train_df[(train_df['event_code']==4100) & (train_df['type'] == 'Assessment')]
dropping_in_data=[
    'installation_id','game_session', 'event_id', 'squeeze_target', \
    'type', 'event_code', 'event_data', 'timestamp', 'date', \
    'game_time', 'event_count', 'game_session|installation_id', 'world']
#correctとincorrectを予測したほうがいいかも
dropping_in_labels=['game_session','installation_id','title','num_correct','num_incorrect', 'accuracy']
print(train.dtypes)
print(dropping_in_data)
print(dropping_in_labels)


event_id           object
game_session       object
timestamp          object
event_data         object
installation_id    object
event_count         int64
event_code          int64
game_time           int64
title              object
type               object
world              object
dtype: object
['installation_id', 'game_session', 'event_id', 'squeeze_target', 'type', 'event_code', 'event_data', 'timestamp', 'date', 'game_time', 'event_count', 'game_session|installation_id', 'world']
['game_session', 'installation_id', 'title', 'num_correct', 'num_incorrect', 'accuracy']


In [5]:
new_train, new_test = transform(train, test)
if train['event_id'].all() == new_train['event_id'].all():
    print('transforming is done!')


experienced_assessment:
100%|██████████| 12629/12629 [01:03<00:00, 199.17it/s]
transforming is done!


In [6]:
sq_new_train = data_squeeze(new_train,mode='train')


data_squeeze:
100%|██████████| 21239/21239 [01:14<00:00, 285.27it/s]


In [7]:
labels = pd.read_csv('./input/train_labels.csv')
labels['game_session|installation_id'] = labels['game_session'] +'|'+ labels['installation_id']
dropped_labels = labels.drop(columns=dropping_in_labels, axis=1)

sq_new_train['game_session|installation_id'].head()
merged_sq_new_train = pd.merge(sq_new_train, dropped_labels, on='game_session|installation_id')
dropped_sq_new_train = merged_sq_new_train.drop(columns=dropping_in_data, axis=1)


In [8]:
dropped_sq_new_train.tail(10)


,title,hour,exp_assess,accuracy_group
17680,4,21,0,1
17681,4,21,1,3
17682,0,21,0,3
17683,2,1,0,3
17684,1,1,0,3
17685,3,12,0,3
17686,2,1,0,3
17687,2,15,0,1
17688,4,1,0,0
17689,2,2,1,1


In [9]:
sq_new_test = data_squeeze(new_test, mode='test')
dropped_sq_new_test = sq_new_test.drop(columns=dropping_in_data, axis=1)

data_squeeze:
test_squeeze:
100%|██████████| 1000/1000 [00:02<00:00, 492.99it/s]


In [10]:
train = dropped_sq_new_train
test = dropped_sq_new_test


In [11]:
len(sub)
len(test)

1000

In [12]:

usefull_features=['title', 'hour', 'exp_assess']
categoricals = ['title', 'hour']
def run_lgb(train, test, usefull_features):
    kf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(train), 4))
    y_pred = np.zeros((len(test), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train, train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = train[usefull_features].iloc[tr_ind], train[usefull_features].iloc[val_ind]
        y_train, y_val = train[target][tr_ind], train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'learning_rate': 0.01,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.75,
            'subsample': 0.75,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 1000000, early_stopping_rounds = 50, 
                          valid_sets=[train_set, val_set], verbose_eval = 100)
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(test[usefull_features]) / 5
    loss_score = cohen_kappa_score(train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    result = pd.Series(np.argmax(oof_pred, axis = 1))
    print('Our oof cohen kappa score is: ', loss_score)
    print(result.value_counts(normalize = True))
    return y_pred
prediction = run_lgb(train, test, usefull_features)
sub['accuracy_group'] = np.round(prediction).astype(int)
sub.to_csv('submission.csv', index = False)
print('done!')



Fold 1
[100]	training's multi_logloss: 1.13414	valid_1's multi_logloss: 1.14665
[200]	training's multi_logloss: 1.09864	valid_1's multi_logloss: 1.1197
[300]	training's multi_logloss: 1.08313	valid_1's multi_logloss: 1.11096
[400]	training's multi_logloss: 1.07481	valid_1's multi_logloss: 1.10786
[500]	training's multi_logloss: 1.07031	valid_1's multi_logloss: 1.10752
Fold 2
[100]	training's multi_logloss: 1.13595	valid_1's multi_logloss: 1.14189
[200]	training's multi_logloss: 1.10115	valid_1's multi_logloss: 1.11235
[300]	training's multi_logloss: 1.08586	valid_1's multi_logloss: 1.10173
[400]	training's multi_logloss: 1.07759	valid_1's multi_logloss: 1.09759
[500]	training's multi_logloss: 1.07272	valid_1's multi_logloss: 1.097
Fold 3
[100]	training's multi_logloss: 1.13487	valid_1's multi_logloss: 1.14501
[200]	training's multi_logloss: 1.0995	valid_1's multi_logloss: 1.11708
[300]	training's multi_logloss: 1.08392	valid_1's multi_logloss: 1.10674
[400]	training's multi_logloss: 1.